# Подготовка данных:

Так как входные данные состоят из массивов (512, 512, 3), то обучение данных затратит больших вычислительных можностей. Именно поэтому мы отрезайзим входную фотографи и маску в размер (256, 256). Так как данных много и они большого размера, то лучше не загружать их всех в оперативную память, а делать это порциями. Кроме того, нормируем наши данные на 255 для улучшения обучения модели. Все это реализуем в генераторе keras_generator:

In [8]:
def keras_generator(path, batch_size):

        
    while True:
        images = os.listdir(f"{path}/images")
        annotations = json.load(open(f"{path}/coco_annotations.json", "r"))
        x_batch = []
        y_batch = []
        
        for i in range(batch_size):
            img_id = int(np.random.choice(images).split(".")[0])
            img = np.array(Image.open(f"{path}/images/{img_id:08}.jpg"))
            mask = get_mask(img_id, annotations)
            img = cv2.resize(img, (256, 256))
            mask = cv2.resize(mask, (256, 256))
            
            x_batch += [img]
            y_batch += [mask]

        x_batch = np.array(x_batch) / 255.
        y_batch = np.array(y_batch) /255.

        yield x_batch, np.expand_dims(y_batch, -1)

Аналогично напишем функции для загрузки валидационных данных и тестовых:

In [10]:
# загрузка валидационных данных для проверки
def load_data(path, batch_size):

    annotations = json.load(open(f"{path}/coco_annotations.json", "r"))    
    while True:
        x_batch = []
        y_batch = []
        for i in range(batch_size):
            img_id = i
            img = np.array(Image.open(f"{path}/images/{img_id:08}.jpg"))
            mask = get_mask(img_id, annotations)
            img = cv2.resize(img, (256, 256))
            
            x_batch += [img]
            y_batch += [mask]
        x_batch = np.array(x_batch) / 255.

        return x_batch, y_batch

In [11]:
# загрузка тестовых данных для проверки
def get_test(path, batch_size):
    x_batch = []
    for i in range(batch_size):
        img_id = i
        img = np.array(Image.open(f"{path}/real_test/images/{img_id:04}.jpg"))
        img = cv2.resize(img, (256, 256))

        x_batch += [img]

    x_batch = np.array(x_batch) / 255.
    return x_batch

в звязи с тем, что мы работаем с изображениями 256x256, нам необходимо их преобразовать обратно в 512x512:

In [ ]:
# конвектируем маску в 512x512
def det_mask(pred, val):
    A= np.zeros((256, 256))
    for i in range(256):
        for j in range(256):
            if (pred[i,j,0] > val):
                A[i,j]=1
    A = cv2.resize(A, (512, 512))
    
    for i in range(512):
        for j in range(512):
            if (A[i,j] != 1):
                A[i,j]=0
    return A

Из-за некорректного сохранения изображений пришлось писать свою немного измененную функцию зохранения:
    

In [13]:
#кастомная функция для сохранения изображений, связвнная с роботой модели на изображениях (256x256)
def save_html_new(paths_to_imgs, pred_masks, path_to_save="results/test"):
    
    paths_to_imgs = np.array(paths_to_imgs)
    pred_masks = np.array(pred_masks)

    if not os.path.exists(path_to_save):
        os.makedirs(path_to_save)

    order = np.argsort(paths_to_imgs)
    paths_to_imgs = paths_to_imgs[order]
    pred_masks = pred_masks[order]

    for path_to_img, pred_mask in zip(paths_to_imgs, pred_masks):
        img_id = path_to_img.split("/")[-1].split(".")[0]
        img = np.array(Image.open(path_to_img))[:, :, :3]
        Image.fromarray(img).save(f"{path_to_save}/{img_id}_img.jpg")
        rle_mask = encode_rle(pred_mask)
        decoded_mask = decode_rle(rle_mask)
        Image.fromarray(decoded_mask).save(f"{path_to_save}/{img_id}_pred_mask.png")
        crop_img = img.copy()
        crop_img[pred_mask == 0] = 0
        Image.fromarray(crop_img).save(f"{path_to_save}/{img_id}_crop.png")

# Аугментации:

Для улучшения обучения модели помимо входные данные будим их также изменять пораждая тем дополнительный датасет для обучегия.
 Реализовано в функции strong_aug:

In [14]:
def strong_aug(p=1.0):
    return Compose([
        ShiftScaleRotate(shift_limit=0.125, scale_limit=0.2, rotate_limit=10, p=0.7, border_mode=cv2.BORDER_CONSTANT),
        RandomCrop(256, 256),
        #PadIfNeeded(min_height=224, min_width=224, border_mode=cv2.BORDER_CONSTANT, p=1.0),
        #Resize(64, 64),
        #RandomRotate90(),
        ElasticTransform(1.), 
        #HorizontalFlip(),
        #Cutout(p=1.),
        #Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.3),
        OneOf([
            MotionBlur(p=.4),
            MedianBlur(blur_limit=3, p=0.3),
            Blur(blur_limit=3, p=0.3),
        ], p=0.5),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=0.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.5),
        OneOf([
            CLAHE(clip_limit=3),
            IAASharpen(),
            IAAEmboss(),
            RandomContrast(),
            RandomBrightness(),
        ], p=0.4),
        HueSaturationValue(p=0.7),
         
        
        
    ],
        p=p)